## Merging of Per-Core Logs
#### Can we assume that if all cores of a running experiment do similar work, we can assume that their correlation matrices are also very similar and hence can be averaged out into 1 matrix.

In [111]:
import os
import pandas as pd
import numpy as np

In [112]:
# get all itrs in a logs directory
def list_itrs(rdtsc_dirname):
    itrs = []
    for file in os.listdir(rdtsc_dirname):
        tags = file.split('_')
        itr = tags[1]
        itrs.append(int(itr))
    itrs = list(set(itrs))
    return itrs

### Test: qps = [200k, 400k, 600k], dvfs = [0xd00, 0x1d00], itr = [all_vals]

In [113]:
import eigen_analysis

cols = eigen_analysis.LINUX_COLS
time_unit = eigen_analysis.TIME_CONVERSION_khz
joules_unit = eigen_analysis.JOULE_CONVERSION

In [114]:
qpss = ['200k', '400k', '600k']
dvfss = ['0xd00', '0x1d00']
run = '0'
rapl = '135'

In [115]:
# for qps in qpss:

#     qps_dir = qps + '_qps/'
#     print(qps_dir)

#     for dvfs in dvfss:
#         dvfs_dir = qps_dir + 'linux_mcd_dmesg_' + run + '_' + dvfs + '_' + rapl + '_' + qps + '/'
#         rdtsc_dir = qps_dir + 'linux_mcd_rdtsc_' + run + '_' + dvfs + '_' + rapl + '_' + qps + '/'
#         print(dvfs_dir)
#         print(rdtsc_dir)

#         # get all itrs tested with this dvfs
#         itrs = list_itrs(dvfs_dir)
#         for itr in itrs:
#             # for syncing log file with recorded experiment time
#             rdtsc_file = rdtsc_dir + 'linux.mcd.rdtsc.' + run + '_' + str(itr) + '_' + dvfs + '_' + rapl + '_' + qps[0:-1] + '000'
#             start, end = eigen_analysis.get_rdtsc(rdtsc_file)
            
#             # to store per-core dfs for subsequent averaging
#             corr_list_no_diffs = []
#             corr_list_diffs = []
            
#             for core in range(0,16):
#                 log_file = dvfs_dir + 'linux.mcd.dmesg.' + run + '_' + str(core) + '_' + str(itr) + '_' + dvfs + '_' + rapl + '_' + qps[0:-1] + '000'
#                 df = pd.read_csv(log_file, sep= ' ', names = cols, index_col='i')

#                 df_no_diffs = df[['rx_bytes', 'rx_desc', 'tx_bytes', 'tx_desc']].copy()
                
#                 df = df[['instructions', 'cycles', 'ref_cycles', 'llc_miss', 'joules', 'timestamp']]
#                 df = df[(df['timestamp'] >= start) & (df['timestamp'] <= end)]
#                 df['timestamp'] = df['timestamp'] - df['timestamp'].min()
#                 df['timestamp'] = df['timestamp'] * time_unit
#                 df['joules'] = df['joules'] * joules_unit
                
#                 # removing empty/zero log-entries
#                 df_copy = df[(df['joules'] > 0) & (df['instructions'] > 0) & (df['cycles'] > 0) \
#                              & (df['ref_cycles'] > 0) & (df['llc_miss'] > 0)].copy()
#                 tmp = df_copy.diff().copy()
#                 tmp.columns = [f'{c}_diff' for c in tmp.columns]
                
#                 # ignoring all log files that produce negative diffs due to overflow
#                 df_diffs_neg = tmp[(tmp['joules_diff'] < 0) | (tmp['instructions_diff'] < 0) | (tmp['cycles_diff'] < 0) \
#                                    | (tmp['ref_cycles_diff'] < 0) | (tmp['llc_miss_diff'] < 0) | (tmp['timestamp_diff'] < 0)]
#                 tmp = tmp[(tmp['joules_diff'] >= 0) | (tmp['instructions_diff'] >= 0) | (tmp['cycles_diff'] >= 0) \
#                                    | (tmp['ref_cycles_diff'] >= 0) | (tmp['llc_miss_diff'] >= 0) | (tmp['timestamp_diff'] >= 0)]
#                 if df_diffs_neg.shape[0] > 0:
#                     print('NEGATIVE DIFFS IN LOG FILE ', log_file)
#                     print(df_diffs_neg)
#                     break

#                 df_diffs = tmp.copy()

#                 # add dfs to list of <= 16 dfs, 1 per core
#                 corr_list_no_diffs.append(df_no_diffs.corr())
#                 corr_list_diffs.append(df_diffs.corr())
                
#             print()
#             print('---------------------------------------- PARSED 16 LOGS ----------------')
#             print()
#             no_diffs_list_len = len(corr_list_no_diffs)
#             diffs_list_len = len(corr_list_diffs)
#             no_diffs_sum = np.zeros((corr_list_no_diffs[0].shape[0], corr_list_no_diffs[0].shape[1]), dtype=np.float64)
#             diffs_sum = np.zeros((corr_list_diffs[0].shape[0], corr_list_diffs[0].shape[1]), dtype=np.float64)
#             for c1 in corr_list_no_diffs:
#                 corr_no_diffs_sum += c1.to_numpy()
#             for c2 in corr_list_diffs:
#                 corr_diffs_sum += c2.to_numpy()
#             corr_no_diffs_avg = corr_no_diffs_sum / corr_no_diffs_len
#             corr_diffs_avg = corr_diffs_sum / corr_diffs_len
            
#             print(corr_no_diffs_avg)
#             print(corr_diffs_avg)
#             break
#         print()
#         print('---------------------------------------- PARSED 12 ITRS ----------------')
#         print()
#         break
#     print()
#     print('---------------------------------------- PARSED 2 DVFSS ----------------')
#     print()
#     break
    

In [116]:
rdtsc_dir = '200k_qps/linux_mcd_rdtsc_0_0xd00_135_200k/'
logs_dir = '200k_qps/linux_mcd_dmesg_0_0xd00_135_200k/'
itrs = list_itrs(rdtsc_dir)
print(itrs)

[350, 2, 100, 40, 200, 10, 300, 400, 50, 20, 250, 30]


In [130]:
df_itr_list = {}
for itr in itrs:
    print('ITR-DELAY = ' + str(itr))
    df_full = pd.DataFrame()

    rdtsc_file = rdtsc_dir + 'linux.mcd.rdtsc.0_' + str(itr) + '_0xd00_135_200000'
    start, end = eigen_analysis.get_rdtsc(rdtsc_file)

    for c in range(0,16):
        file = logs_dir + 'linux.mcd.dmesg.0_' + str(c) + '_' + str(itr) + '_0xd00_135_200000'
        df = pd.read_csv(file, sep = ' ', names = cols, index_col='i').drop(['c1', 'c1e', 'c3', 'c6', 'c7', \
                                                                             'rx_bytes', 'rx_desc', 'tx_bytes', 'tx_desc'], axis=1)
        df = df[(df['timestamp'] >= start) & (df['timestamp'] <= end)]
        df['timestamp'] = df['timestamp'] - df['timestamp'].min()
        df['timestamp'] = df['timestamp'] * time_unit
        df['joules'] = df['joules'] * joules_unit

        # removing empty/zero log-entries
        # -> these represent interrupt occurrences at a frequency larger than ~1ms
        df_copy = df[(df['joules'] > 0) & (df['instructions'] > 0) & (df['cycles'] > 0) \
                 & (df['ref_cycles'] > 0) & (df['llc_miss'] > 0)].copy()
        # computing diffs
        tmp = df_copy.diff().dropna()
        tmp.columns = [f'{c}_diff' for c in tmp.columns]

        # isolating log entries with negative diffs due to register overflow
        df_diffs_neg = tmp[(tmp['joules_diff'] < 0) | (tmp['instructions_diff'] < 0) | (tmp['cycles_diff'] < 0) \
                       | (tmp['ref_cycles_diff'] < 0) | (tmp['llc_miss_diff'] < 0) | (tmp['timestamp_diff'] < 0)]
        if df_diffs_neg.shape[0] > 0:
            print('NEGATIVE DIFFS')
            print(df_diffs_neg)

        # TODO 
        df_diffs = tmp[(tmp['joules_diff'] >= 0) | (tmp['instructions_diff'] >= 0) | (tmp['cycles_diff'] >= 0) \
                       | (tmp['ref_cycles_diff'] >= 0) | (tmp['llc_miss_diff'] >= 0) | (tmp['timestamp_diff'] >= 0)]

        # dropping timestamp_diff column - subject to change
        df_diffs = df_diffs.drop(['timestamp_diff'], axis=1).reset_index()

        # merge per-core dataframes
        # SCHEME 1: merge on order of occurrence per-core
        if df_full.shape[0] == 0:
            df_full = df_diffs.copy()
        else:
            df_full = df_full.merge(df_diffs, left_index=True, right_index=True, how='outer', \
                                    sort=True, suffixes=('', '_0')).fillna(0)

        # total entries ~= (20 s * 1000 ms/s * 1000 us/ms )/ ITR us
        print(df.shape[0], df_copy.shape[0], tmp.shape[0], df_diffs.shape[0])
        #break
        
    print()
    print('---------------------------------------- PARSED 16 LOGS ----------------')
    print()
    df_itr_list[str(itr)] = df_full
    #break
print()
print('---------------------------------------- PARSED 12 ITRS ----------------')
print()

ITR-DELAY = 350
55812 18604 18603 18603
55808 18603 18602 18602
55813 18604 18603 18603
55803 18602 18601 18601
55815 18605 18604 18604
55808 18603 18602 18602
55816 18606 18605 18605
NEGATIVE DIFFS
     instructions_diff  cycles_diff  ref_cycles_diff  llc_miss_diff  \
i                                                                     
376        -98514712.0 -251175477.0     -562260700.0     -6299188.0   

     joules_diff  timestamp_diff  
i                                 
376   -299.33798        0.000358  
55808 18604 18603 18603
55812 18604 18603 18603
55806 18602 18601 18601
55814 18605 18604 18604
55809 18603 18602 18602
55813 18604 18603 18603
55805 18602 18601 18601
55813 18604 18603 18603
55805 18603 18602 18602

---------------------------------------- PARSED 16 LOGS ----------------

ITR-DELAY = 2
733395 19312 19311 19311
732547 19300 19299 19299
733689 19309 19308 19308
735630 19310 19309 19309
735190 19312 19311 19311
736170 19311 19310 19310
739786 19328 19327 19327
73

In [133]:
df_itr_list['100']

,i,instructions_diff,cycles_diff,ref_cycles_diff,llc_miss_diff,joules_diff,i_0,instructions_diff_0,cycles_diff_0,ref_cycles_diff_0,...,cycles_diff_0,ref_cycles_diff_0,llc_miss_diff_0,joules_diff_0,i_0,instructions_diff_0,cycles_diff_0,ref_cycles_diff_0,llc_miss_diff_0,joules_diff_0
0,87.0,494620.0,1156267.0,2579347.0,2857.0,0.107299,143.0,547085.0,1166322.0,2612987.0,...,1205892.0,2698856.0,3364.0,0.107299,86.0,522229.0,1247367.0,2784609.0,3465.0,0.112057
1,97.0,480904.0,910190.0,2030377.0,2506.0,0.115168,153.0,460793.0,1079236.0,2407551.0,...,807145.0,1800639.0,1742.0,0.115168,96.0,466145.0,890409.0,1986297.0,2448.0,0.115290
2,107.0,453412.0,904490.0,2017791.0,2402.0,0.234057,163.0,417253.0,827774.0,1846546.0,...,697103.0,1555096.0,1344.0,0.118645,106.0,424304.0,820126.0,1829581.0,2191.0,0.120109
3,117.0,419248.0,884458.0,1973044.0,2205.0,0.000000,173.0,297719.0,781661.0,1743741.0,...,810409.0,1807947.0,2051.0,0.115412,116.0,374160.0,694892.0,1550253.0,1685.0,0.112789
4,127.0,332785.0,668349.0,1491035.0,1564.0,0.235704,183.0,577270.0,1037151.0,2313649.0,...,621149.0,1385765.0,1646.0,0.120963,126.0,392370.0,769997.0,1717844.0,1683.0,0.124440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19250,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
19251,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
19252,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
19253,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000


In [135]:
for itr in df_itr_list.keys():
    df_itr_list[itr] = df_itr_list[itr].drop(['i', 'i_0'], axis=1)

In [137]:
df_itr_list['350']

,instructions_diff,cycles_diff,ref_cycles_diff,llc_miss_diff,joules_diff,instructions_diff_0,cycles_diff_0,ref_cycles_diff_0,llc_miss_diff_0,joules_diff_0,...,instructions_diff_0,cycles_diff_0,ref_cycles_diff_0,llc_miss_diff_0,joules_diff_0,instructions_diff_0,cycles_diff_0,ref_cycles_diff_0,llc_miss_diff_0,joules_diff_0
0,399537.0,990997.0,2210670.0,2764.0,0.105286,520473.0,1167784.0,2605070.0,3700.0,0.000000,...,529876.0,1114095.0,2485271.0,3127.0,0.105286,331939.0,858689.0,1915508.0,2468.0,0.089853
1,445229.0,987064.0,2201970.0,2381.0,0.124745,489310.0,1103770.0,2462274.0,3118.0,0.235765,...,480342.0,1020831.0,2277225.0,2295.0,0.243268,431164.0,1012455.0,2258549.0,2685.0,0.235765
2,452146.0,1042160.0,2324785.0,2441.0,0.118523,404601.0,831156.0,1854115.0,1917.0,0.118767,...,375781.0,752779.0,1679825.0,2042.0,0.116022,428884.0,909838.0,2029623.0,2531.0,0.118767
3,507916.0,936175.0,2088522.0,2147.0,0.116022,362481.0,926263.0,2066221.0,2080.0,0.117913,...,353457.0,672659.0,1500663.0,1950.0,0.113521,357663.0,767242.0,1716713.0,1958.0,0.117913
4,435888.0,758646.0,1692411.0,1762.0,0.113521,511304.0,880974.0,1965272.0,2652.0,0.110715,...,428203.0,777841.0,1735244.0,1996.0,0.113704,376845.0,701222.0,1564347.0,2100.0,0.110715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18600,446134.0,880526.0,1964779.0,1840.0,0.108824,532637.0,927019.0,2067961.0,2181.0,0.124745,...,435597.0,723098.0,1613096.0,1425.0,0.108824,622547.0,1128650.0,2517780.0,2949.0,0.124745
18601,683097.0,1225726.0,2734323.0,2798.0,0.127307,623175.0,1292325.0,2882890.0,2753.0,0.123830,...,411970.0,608056.0,1356446.0,1162.0,0.127307,774626.0,1396600.0,3115499.0,3511.0,0.123830
18602,578069.0,1038438.0,2316520.0,2152.0,0.116937,0.0,0.0,0.0,0.0,0.000000,...,308809.0,459849.0,1025846.0,850.0,0.116937,0.0,0.0,0.0,0.0,0.000000
18603,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000


In [80]:
#np.average(df_full[['instructions_diff', 'instructions_diff_0']].loc[0].to_numpy())

In [139]:
df_itr_merged_list = {}
for itr in df_itr_list.keys():
    # creating a larger and more sparse dataframe of average diffs across log files
    df_final = pd.DataFrame(columns=['instructions_diff', 'cycles_diff', 'ref_cycles_diff', \
                                     'llc_miss_diff', 'joules_diff'])
    for i,j in df_itr_list[itr].iterrows():
        new_row = {}
        for col in df_final.columns:
            row = df_itr_list[itr][[col, col+'_0']].loc[i]
            avg = np.average(row.to_numpy())
            new_row[col] = avg
        df_final = df_final._append(new_row, ignore_index = True)
    print(df_final)
    df_itr_merged_list[itr] = df_final 
    # TODO joules_sum averages to per-core joules_sum as a test

       instructions_diff   cycles_diff  ref_cycles_diff  llc_miss_diff  \
0            389777.3750  9.962972e+05     2.222783e+06      2590.2500   
1            449189.9375  1.005234e+06     2.242472e+06      2615.3750   
2            444722.6250  8.569940e+05     1.912113e+06      2217.8125   
3            391786.3125  7.666286e+05     1.713039e+06      1982.6875   
4            413895.3125  7.385942e+05     1.647678e+06      1982.5000   
...                  ...           ...              ...            ...   
18600        444806.0625  8.590969e+05     1.918415e+06      2036.8125   
18601        444609.0625  8.226677e+05     1.836624e+06      1994.5000   
18602        323394.0625  5.900635e+05     1.316310e+06      1415.3125   
18603        117732.4375  2.261037e+05     5.043916e+05       524.7500   
18604         26356.7500  4.796988e+04     1.072764e+05       100.5000   

       joules_diff  
0         0.091393  
1         0.180575  
2         0.118176  
3         0.123418  
4     

       instructions_diff   cycles_diff  ref_cycles_diff  llc_miss_diff  \
0            430412.1250  1.086684e+06     2.428587e+06      2810.6250   
1            477289.0000  9.787229e+05     2.183352e+06      2513.1875   
2            423600.0625  8.662191e+05     1.932393e+06      2217.0000   
3            401001.5625  8.129679e+05     1.813598e+06      2019.7500   
4            420963.3125  8.417258e+05     1.877750e+06      2040.7500   
...                  ...           ...              ...            ...   
19267         31539.1250  6.528862e+04     1.456543e+05        68.5000   
19268         33388.9375  7.496981e+04     1.672466e+05       132.5000   
19269         78867.0625  1.525877e+05     3.403911e+05       291.3750   
19270         71471.2500  1.450448e+05     3.235657e+05       271.8125   
19271         35105.8750  7.999088e+04     1.784406e+05       154.5625   

       joules_diff  
0         0.106857  
1         0.123754  
2         0.119895  
3         0.125500  
4     

In [140]:
df_itr_merged_list

{'350':        instructions_diff   cycles_diff  ref_cycles_diff  llc_miss_diff  \
 0            389777.3750  9.962972e+05     2.222783e+06      2590.2500   
 1            449189.9375  1.005234e+06     2.242472e+06      2615.3750   
 2            444722.6250  8.569940e+05     1.912113e+06      2217.8125   
 3            391786.3125  7.666286e+05     1.713039e+06      1982.6875   
 4            413895.3125  7.385942e+05     1.647678e+06      1982.5000   
 ...                  ...           ...              ...            ...   
 18600        444806.0625  8.590969e+05     1.918415e+06      2036.8125   
 18601        444609.0625  8.226677e+05     1.836624e+06      1994.5000   
 18602        323394.0625  5.900635e+05     1.316310e+06      1415.3125   
 18603        117732.4375  2.261037e+05     5.043916e+05       524.7500   
 18604         26356.7500  4.796988e+04     1.072764e+05       100.5000   
 
        joules_diff  
 0         0.091393  
 1         0.180575  
 2         0.118176  
 3 

In [141]:
for itr in df_itr_merged_list.keys():
    outfile = 'df_itr_' + itr + '_merged_cores'
    df_itr_merged_list[itr].to_csv(outfile)

In [142]:
for itr in df_itr_merged_list.keys():
    print(df_itr_merged_list[itr].corr())

                   instructions_diff  cycles_diff  ref_cycles_diff  \
instructions_diff           1.000000     0.991440         0.991129   
cycles_diff                 0.991440     1.000000         0.999981   
ref_cycles_diff             0.991129     0.999981         1.000000   
llc_miss_diff               0.961110     0.982967         0.983212   
joules_diff                 0.957427     0.979806         0.980070   

                   llc_miss_diff  joules_diff  
instructions_diff       0.961110     0.957427  
cycles_diff             0.982967     0.979806  
ref_cycles_diff         0.983212     0.980070  
llc_miss_diff           1.000000     0.998898  
joules_diff             0.998898     1.000000  
                   instructions_diff  cycles_diff  ref_cycles_diff  \
instructions_diff           1.000000     0.888553         0.888531   
cycles_diff                 0.888553     1.000000         1.000000   
ref_cycles_diff             0.888531     1.000000         1.000000   
llc_miss_di

In [28]:
# start, end = eigen_analysis.get_rdtsc(rdtsc)
# df1 = df1[(df1['timestamp'] >= start) & (df1['timestamp'] <= end)]
# df2 = df2[(df2['timestamp'] >= start) & (df2['timestamp'] <= end)]
# df1['timestamp'] = df1['timestamp'] - df1['timestamp'].min()
# df1['timestamp'] = df1['timestamp'] * time_unit
# df1['joules'] = df1['joules'] * joules_unit
# df2['timestamp'] = df2['timestamp'] - df2['timestamp'].min()
# df2['timestamp'] = df2['timestamp'] * time_unit
# df2['joules'] = df2['joules'] * joules_unit

In [29]:
# # removing empty/zero log-entries
# df1_copy = df1[(df1['joules'] > 0) & (df1['instructions'] > 0) & (df1['cycles'] > 0) \
#              & (df1['ref_cycles'] > 0) & (df1['llc_miss'] > 0)].copy()
# df2_copy = df2[(df2['joules'] > 0) & (df2['instructions'] > 0) & (df2['cycles'] > 0) \
#              & (df2['ref_cycles'] > 0) & (df2['llc_miss'] > 0)].copy()

# # computing diffs
# tmp1 = df1_copy.diff().dropna()
# tmp1.columns = [f'{c}_diff' for c in tmp1.columns]
# tmp2 = df2_copy.diff().dropna()
# tmp2.columns = [f'{c}_diff' for c in tmp2.columns]

In [35]:
#tmp1

In [34]:
# # ignoring all log files that produce negative diffs due to overflow
# df1_diffs_neg = tmp1[(tmp1['joules_diff'] < 0) | (tmp1['instructions_diff'] < 0) | (tmp1['cycles_diff'] < 0) \
#                    | (tmp1['ref_cycles_diff'] < 0) | (tmp1['llc_miss_diff'] < 0) | (tmp1['timestamp_diff'] < 0)]
# df1_diffs = tmp1[(tmp1['joules_diff'] >= 0) | (tmp1['instructions_diff'] >= 0) | (tmp1['cycles_diff'] >= 0) \
#                    | (tmp1['ref_cycles_diff'] >= 0) | (tmp1['llc_miss_diff'] >= 0) | (tmp1['timestamp_diff'] >= 0)]
# if df1_diffs_neg.shape[0] > 0:
#     print('NEGATIVE DIFFS')
#     print(df1_diffs_neg)

# df2_diffs_neg = tmp2[(tmp2['joules_diff'] < 0) | (tmp2['instructions_diff'] < 0) | (tmp2['cycles_diff'] < 0) \
#                    | (tmp2['ref_cycles_diff'] < 0) | (tmp2['llc_miss_diff'] < 0) | (tmp2['timestamp_diff'] < 0)]
# df2_diffs = tmp2[(tmp2['joules_diff'] >= 0) | (tmp2['instructions_diff'] >= 0) | (tmp2['cycles_diff'] >= 0) \
#                    | (tmp2['ref_cycles_diff'] >= 0) | (tmp2['llc_miss_diff'] >= 0) | (tmp2['timestamp_diff'] >= 0)]
# if df2_diffs_neg.shape[0] > 0:
#     print('NEGATIVE DIFFS')
#     print(df2_diffs_neg)
    
# # dropping timestamp_diff column
# df1_diffs = df1_diffs.drop(['timestamp_diff'], axis=1).reset_index()
# df2_diffs = df2_diffs.drop(['timestamp_diff'], axis=1).reset_index()

In [37]:
#df2_diffs

In [38]:
#df1_diffs

In [14]:
#df11 = df1_diffs.head(10)
#df22 = df2_diffs.head(10)

In [39]:
# df11 = df1_diffs.copy()
# df22 = df2_diffs.copy()

In [40]:
# df22

In [42]:
#df_full = df11.merge(df22, left_index=True, right_index=True, how='outer', sort=True).fillna(0).copy()

In [43]:
#df_full

In [84]:
# # creating a larger and more sparse dataframe of average diffs across log files
# df_final = pd.DataFrame(columns=['instructions_diff', 'cycles_diff', 'ref_cycles_diff', 'llc_miss_diff', 'joules_diff'])
# count = 0
# for i,j in df_full.iterrows():
#     new_row = {}
#     for col in df_final.columns:
#         sum = 0
#         for s in ['x', 'y']:
#             sum += j[col+'_'+s]
#         new_row[col] = sum / 2
#     df_final = df_final._append(new_row, ignore_index = True)
#     count += 1

# # TODO joules_sum averages to per-core joules_sum as a test

In [83]:
#df_final

In [85]:
# df_final.corr()